In [9]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from sklearn.ensemble import GradientBoostingRegressor
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
import warnings
import logging

# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
logging.getLogger('cmdstanpy').setLevel(logging.CRITICAL)
logging.getLogger('prophet').setLevel(logging.CRITICAL)

# Load the dataset
df = pd.read_csv('Trueliquor_store_sales_data.csv')

# Ensure the 'Date' column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Feature Engineering: Adding Day of Week, Month, and Is Weekend as Features
df['day_of_week'] = df['Date'].dt.dayofweek
df['month'] = df['Date'].dt.month
df['quarter'] = df['Date'].dt.quarter
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
df['is_holiday'] = df['Date'].isin(pd.to_datetime(['2017-12-25', '2017-12-31'])).astype(int)

# Create lag features
for lag in range(1, 8):  # 1 to 7 days lag
    df[f'lag_{lag}'] = df['Total Amount'].shift(lag)

# Create rolling window features
df['rolling_mean_7'] = df['Total Amount'].rolling(window=7).mean()
df['rolling_std_7'] = df['Total Amount'].rolling(window=7).std()

# Drop rows with NaN values after feature engineering
df.dropna(inplace=True)

# Prepare data for modeling
df_model = df[['Date', 'Total Amount', 'day_of_week', 'month', 'quarter', 'is_weekend', 'is_holiday', 
               'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 
               'rolling_mean_7', 'rolling_std_7']]

# Ensure the 'Date' column is in datetime format
df_model['Date'] = pd.to_datetime(df_model['Date'])

# Use TimeSeriesSplit for cross-validation
tscv = TimeSeriesSplit(n_splits=10)
train_val_sets = [(df_model.iloc[train_index].copy(), df_model.iloc[val_index].copy()) for train_index, val_index in tscv.split(df_model)]

# Define model training functions
def train_arima(train_df):
    model = ARIMA(train_df['Total Amount'], order=(5, 1, 0))
    model_fit = model.fit()
    return model_fit

def train_prophet(train_df):
    model = Prophet()
    model.fit(train_df.rename(columns={'Date': 'ds', 'Total Amount': 'y'}))
    return model

def train_xgboost(train_df):
    model = GradientBoostingRegressor()
    X_train = train_df.drop(columns=['Total Amount', 'Date'])
    y_train = train_df['Total Amount']
    model.fit(X_train, y_train)
    return model

def train_lstm(train_df):
    X_train = train_df.drop(columns=['Total Amount', 'Date']).values
    y_train = train_df['Total Amount'].values
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)
    return model

# Define model forecasting functions
def forecast_arima(model, steps):
    forecast = model.forecast(steps)
    return forecast

def forecast_prophet(model, steps):
    future = model.make_future_dataframe(periods=steps)
    forecast = model.predict(future)
    return forecast['yhat'][-steps:].values

def forecast_xgboost(model, val_df):
    X_val = val_df.drop(columns=['Total Amount', 'Date'])
    forecast = model.predict(X_val)
    return forecast

def forecast_lstm(model, val_df):
    X_val = val_df.drop(columns=['Total Amount', 'Date']).values
    X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
    forecast = model.predict(X_val)
    return forecast.flatten()

# Define evaluation function
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    mse = mean_squared_error(y_true, y_pred)
    mbd = np.mean(y_pred - y_true)
    smape = np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true))) * 100
    r_squared = r2_score(y_true, y_pred)
    return mae, rmse, mape, mse, mbd, smape, r_squared

# Perform evaluation for each split for all models
results = { 'ARIMA': [], 'Prophet': [], 'XGBoost': [], 'LSTM': [] }

for i, (train_df, val_df) in enumerate(train_val_sets):
    # Train models for this split
    arima_model = train_arima(train_df)
    prophet_model = train_prophet(train_df)
    xgb_model = train_xgboost(train_df)
    lstm_model = train_lstm(train_df)
    
    # Forecast for the validation period
    arima_forecast = forecast_arima(arima_model, len(val_df))
    prophet_forecast = forecast_prophet(prophet_model, len(val_df))
    xgb_forecast = forecast_xgboost(xgb_model, val_df)
    lstm_forecast = forecast_lstm(lstm_model, val_df)
    
    # Evaluate each model
    models = { 'ARIMA': arima_forecast, 'Prophet': prophet_forecast, 'XGBoost': xgb_forecast, 'LSTM': lstm_forecast }
    
    for model_name, forecast in models.items():
        eval_results = evaluate_model(val_df['Total Amount'], forecast)
        results[model_name].append(eval_results)
        print(f"Split {i+1} - {model_name} - MAE: {eval_results[0]}, RMSE: {eval_results[1]}, MAPE: {eval_results[2]}%, MSE: {eval_results[3]}, MBD: {eval_results[4]}, sMAPE: {eval_results[5]}%, R-squared: {eval_results[6]}")


C:\Users\smart\AppData\Local\Temp\ipykernel_27392\3478460594.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['Date'] = pd.to_datetime(df_model['Date'])


231/231 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Split 1 - ARIMA - MAE: 21.437011934753443, RMSE: 29.43477129371885, MAPE: 58.79650700327181%, MSE: 866.4057611135353, MBD: -6.2957081873190495, sMAPE: 47.83500059562026%, R-squared: -0.04796307941603151
Split 1 - Prophet - MAE: 22.3450514588805, RMSE: 28.872381181179797, MAPE: 72.43596002221749%, MSE: 833.6143950713454, MBD: 1.468606908800427, sMAPE: 49.33247691647082%, R-squared: -0.00830020726284375
Split 1 - XGBoost - MAE: 15.890296794446453, RMSE: 20.41278152413022, MAPE: 49.93258992561375%, MSE: 416.68164955187206, MBD: -0.03250602527641657, sMAPE: 37.09786878875729%, R-squared: 0.49600175322090223
Split 1 - LSTM - MAE: 4.38168347458767, RMSE: 8.675771319343728, MAPE: 11.206408619538367%, MSE: 75.26900798554719, MBD: -1.793538586538526, sMAPE: 10.648325698104358%, R-squared: 0.9089581984176264
231/231 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Split 2 - ARIMA - MAE: 20.82264235299091, RMSE: 28.897510056382508, MAPE: 53.523014604829086%, MSE: 835.0660

In [11]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Load the dataset
df = pd.read_csv('Trueliquor_store_sales_data.csv')

# Ensure the 'Date' column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Feature Engineering: Adding Day of Week, Month, and Is Weekend as Features
df['day_of_week'] = df['Date'].dt.dayofweek
df['month'] = df['Date'].dt.month
df['quarter'] = df['Date'].dt.quarter
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
df['is_holiday'] = df['Date'].isin(pd.to_datetime(['2017-12-25', '2017-12-31'])).astype(int)

# Create lag features
for lag in range(1, 8):  # 1 to 7 days lag
    df[f'lag_{lag}'] = df['Total Amount'].shift(lag)

# Create rolling window features
df['rolling_mean_7'] = df['Total Amount'].rolling(window=7).mean()
df['rolling_std_7'] = df['Total Amount'].rolling(window=7).std()

# Drop rows with NaN values after feature engineering
df.dropna(inplace=True)

# Prepare data for modeling
df_model = df[['Date', 'Total Amount', 'day_of_week', 'month', 'quarter', 'is_weekend', 'is_holiday', 
               'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 
               'rolling_mean_7', 'rolling_std_7']]

# Split into features and target
X_train = df_model.drop(columns=['Date', 'Total Amount']).values
y_train = df_model['Total Amount'].values

# Reshape for LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Define the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
lstm_model.add(LSTM(50))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the LSTM model
lstm_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1)


Epoch 1/20


C:\Users\smart\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2536/2536 ━━━━━━━━━━━━━━━━━━━━ 25s 9ms/step - loss: 1241.9176
Epoch 2/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 42s 9ms/step - loss: 541.8282
Epoch 3/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - loss: 48.5645
Epoch 4/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 39s 8ms/step - loss: 12.9334
Epoch 5/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 41s 8ms/step - loss: 6.9419
Epoch 6/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - loss: 6.1250
Epoch 7/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - loss: 4.8471
Epoch 8/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - loss: 4.4968
Epoch 9/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 40s 8ms/step - loss: 3.6748
Epoch 10/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 42s 9ms/step - loss: 4.1528
Epoch 11/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - loss: 3.9629
Epoch 12/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - loss: 3.6573
Epoch 13/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 26s 10ms/step - loss: 2.9614
Epoch 14/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 24s 9ms/step - loss: 3.2033
Epoch 15/20
2536/2

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from datetime import timedelta
import random
import os

# Load and prepare the dataset
df = pd.read_csv('Trueliquor_store_sales_data.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['day_of_week'] = df['Date'].dt.dayofweek
df['month'] = df['Date'].dt.month
df['quarter'] = df['Date'].dt.quarter
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
df['is_holiday'] = df['Date'].isin(pd.to_datetime(['2017-12-25', '2017-12-31'])).astype(int)

# Create lag features
for lag in range(1, 8):
    df[f'lag_{lag}'] = df['Total Amount'].shift(lag)

df['rolling_mean_7'] = df['Total Amount'].rolling(window=7).mean()
df['rolling_std_7'] = df['Total Amount'].rolling(window=7).std()
df.dropna(inplace=True)

# Prepare data for modeling
df_model = df[['Date', 'Total Amount', 'day_of_week', 'month', 'quarter', 'is_weekend', 'is_holiday',
               'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean_7', 'rolling_std_7']]

X_train = df_model.drop(columns=['Date', 'Total Amount']).values
y_train = df_model['Total Amount'].values

# Reshape for LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Define the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
lstm_model.add(LSTM(50))
lstm_model.add(Dense(1, activation='relu'))  # Use ReLU to ensure non-negative output
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the LSTM model
lstm_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1)

# Generate predictions for 2025
forecast_dates = pd.date_range(start="2025-01-01", end="2025-12-31")
last_row = df_model.drop(columns=['Date']).iloc[-1].values.reshape(1, -1, 1)

predictions = []
for _ in range(len(forecast_dates)):
    prediction = lstm_model.predict(last_row)
    prediction = max(1000, prediction[0, 0])  # Ensure minimum daily revenue of $1,000
    predictions.append(prediction)
    last_row = np.append(last_row[:, 1:, :], np.array(prediction).reshape(1, 1, 1), axis=1)

# Calculate the total predicted revenue
total_predicted_revenue = sum(predictions)

# Scale predictions to target $2 million in total revenue
scaling_factor = 2_000_000 / total_predicted_revenue
scaled_predictions = [pred * scaling_factor for pred in predictions]

# Generate random transactions based on scaled daily revenue
items = [
    {"item": "Red Wine", "brand": "Robert Mondavi", "category": "Wine", "price": 12.99},
    {"item": "Beer Pack", "brand": "Budweiser", "category": "Beer", "price": 18.49},
    {"item": "Vodka", "brand": "Smirnoff", "category": "Spirits", "price": 25.79},
    {"item": "White Wine", "brand": "Barefoot", "category": "Wine", "price": 15.99},
    {"item": "Whiskey", "brand": "Jack Daniel's", "category": "Spirits", "price": 30.99},
    {"item": "Gin", "brand": "Tanqueray", "category": "Spirits", "price": 22.59},
    {"item": "Champagne", "brand": "Moët & Chandon", "category": "Wine", "price": 45.99},
    {"item": "Craft Beer", "brand": "Sierra Nevada", "category": "Beer", "price": 20.49},
    {"item": "Rum", "brand": "Bacardi", "category": "Spirits", "price": 28.79},
    {"item": "Cigarettes", "brand": "Marlboro", "category": "Smoking Products", "price": 10.99},
    {"item": "Cigars", "brand": "Cohiba", "category": "Smoking Products", "price": 15.49},
]

transaction_data = []

for date, total_pred in zip(forecast_dates, scaled_predictions):
    remaining_revenue = total_pred

    while remaining_revenue > 0:
        item = random.choice(items)
        quantity = random.randint(1, 3)
        total_amount = item['price'] * quantity
        if remaining_revenue - total_amount < 0:
            total_amount = remaining_revenue

        payment_method = random.choice(["Cash", "Credit Card", "Mobile Payment"])
        if payment_method == "Credit Card":
            total_amount *= 1.03
        
        margin = f"{round(random.uniform(0.28, 0.35) * 100, 2)}%" if item['category'] == "Spirits" else f"{round(random.uniform(0.50, 0.65) * 100, 2)}%" if item['category'] == "Smoking Products" else f"{round(random.uniform(0.25, 0.30) * 100, 2)}%"

        transaction_data.append([
            date,  # Keeping only Date
            item['item'], item['brand'], item['category'], quantity, item['price'], payment_method, margin, f"${total_amount:.2f}"
        ])
        
        remaining_revenue -= total_amount

# Create DataFrame for transactions
transaction_df = pd.DataFrame(transaction_data, columns=[
    "Date", "Item Purchased", "Brand", "Category", "Quantity", "Price per Unit", "Payment Method", "Margin", "Total Amount"
])

# Save the DataFrame to Excel
excel_filename = '2025_Transaction_Forecast_RJS_INC.xlsx'
transaction_df.to_excel(excel_filename, index=False)

# Display the path where the file is saved
file_path = os.path.abspath(excel_filename)
print(f'The forecast dataset has been saved to {file_path}')


Epoch 1/20


C:\Users\smart\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2536/2536 ━━━━━━━━━━━━━━━━━━━━ 32s 11ms/step - loss: 1276.9056
Epoch 2/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 27s 10ms/step - loss: 691.3575
Epoch 3/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 32s 12ms/step - loss: 72.6309
Epoch 4/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 40s 12ms/step - loss: 17.8146
Epoch 5/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 29s 11ms/step - loss: 7.0668
Epoch 6/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 33s 13ms/step - loss: 5.8570
Epoch 7/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 39s 12ms/step - loss: 4.4400
Epoch 8/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 30s 12ms/step - loss: 4.5233
Epoch 9/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 42s 12ms/step - loss: 3.6818
Epoch 10/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 33s 13ms/step - loss: 3.5669
Epoch 11/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 43s 14ms/step - loss: 3.8253
Epoch 12/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 41s 14ms/step - loss: 3.6448
Epoch 13/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 28s 11ms/step - loss: 3.0002
Epoch 14/20
2536/2536 ━━━━━━━━━━━━━━━━━━━━ 30s 12ms/step - loss: 2.8350
Epoch